In [1]:
# Importing necessary modules
import os
import urllib.request as request
import zipfile

In [2]:
# Changing the current directory to the parent directory
os.chdir("../")

# Getting the current working directory and printing it
%pwd

'c:\\Users\\P52s\\Documents\\Python Project\\ElasticNet_Model'

In [4]:
# Importing constants and utility functions from the WineX package
from WineX.constants import *
from WineX.utils.common import read_yaml, create_directories


In [5]:
from dataclasses import dataclass
from pathlib import Path

# Defining a data class for configuration related to data ingestion
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
# ConfigurationManager class for managing configurations
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        # Reading configuration, parameters, and schema from YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Creating necessary directories
        create_directories([self.config.artifacts_root])

    # Method to get the data ingestion configuration
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # Creating necessary directories
        create_directories([config.root_dir])

        # Creating a DataIngestionConfig object
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
# Importing logger and utility function from WineX package
from WineX import logger
from WineX.utils.common import get_size

In [9]:
# Class for handling data ingestion operations
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # Method to download the data file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # Downloading the file using urllib
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with the following info:\n{headers}")
        else:
            # Logging if the file already exists
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    # Method to extract the zip file
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        # Creating the unzip directory if it doesn't exist
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            # Extracting the contents of the zip file
            zip_ref.extractall(unzip_path)

In [10]:
# Exception handling for the main process
try:
    # Creating ConfigurationManager and obtaining data ingestion configuration
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    # Creating DataIngestion object and performing data download and extraction
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    # Raising an exception if an error occurs
    raise e

[2023-12-09 01:22:36,201: INFO: common] yaml file: config\config.yaml loaded successfully
[2023-12-09 01:22:36,209: INFO: common] yaml file: params.yaml loaded successfully
[2023-12-09 01:22:36,224: INFO: common] yaml file: schema.yaml loaded successfully
[2023-12-09 01:22:36,229: INFO: common] created directory at: artifacts
[2023-12-09 01:22:36,235: INFO: common] created directory at: artifacts/data_ingestion


[2023-12-09 01:22:38,229: INFO: 550461538] artifacts/data_ingestion/data.zip downloaded! with the following info:
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D6CA:E2DAD:C0706A:C8B221:6573B34D
Accept-Ranges: bytes
Date: Sat, 09 Dec 2023 00:22:37 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs1050092-MRS
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1702081357.377840,VS0,VE184
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 2512f70f30fd65d1470a2f896a143c19f370460d
Expires: Sat, 09 Dec 2023 00:27:37 GMT
Source-Age: 0


